In [0]:
storage_account_name = "datalakeintake"
container_name = "pre-processed"
processed_container = "processed"
mount_point = "/mnt/datalake"

client_id = "49c82780-4909-4b56-8c7d-01f18f5a4096"
tenant_id = "a94de956-de2f-426a-a083-b5583df53135"
client_secret = dbutils.secrets.get(scope="key-vault-datalake-databrick",key="databricks-datalake-keyvault-credentials")

#set up the spark configurations
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

source_url = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/"
display(dbutils.fs.ls(source_url))

path,name,size,modificationTime
abfss://pre-processed@datalakeintake.dfs.core.windows.net/product-data/,product-data/,0,1741828301000


In [0]:
sub_folder = "product-data"
folder_path = f"{source_url}/{sub_folder}/"
folders = dbutils.fs.ls(folder_path)
display(folders)

path,name,size,modificationTime
abfss://pre-processed@datalakeintake.dfs.core.windows.net/product-data/olist_order_payments,olist_order_payments,4105147,1741876661000
abfss://pre-processed@datalakeintake.dfs.core.windows.net/product-data/olist_products_dataset,olist_products_dataset,1721338,1741876671000
abfss://pre-processed@datalakeintake.dfs.core.windows.net/product-data/olist_sellers_dataset,olist_sellers_dataset,137840,1741876686000
abfss://pre-processed@datalakeintake.dfs.core.windows.net/product-data/product_category_name_translation,product_category_name_translation,2820,1741876655000


In [0]:
from pyspark.sql.functions import col,to_date,datediff,current_date,when
def clean_datafram(df,name):
    print("Cleaning "+name)
    return df.dropDuplicates().na.drop('all')

def get_volume_utilization(df):
    return df.withColumn(
        "Space_utilization",
        when(
            (col("product_length_cm").cast("int").isNotNull()) & (col("product_height_cm").cast("int").isNotNull()) & (col("product_width_cm").cast("int").isNotNull()) &
            (col("product_length_cm") != 0) & (col("product_height_cm") != 0) & (col("product_width_cm") != 0), 
            (col("product_length_cm") * col("product_height_cm") * col("product_width_cm"))
        ).otherwise(None)
    )

file_path = f"{folder_path}{folders[1].name}"
orders_df = spark.read.format("parquet").option("header", "true").load(file_path)
orders_df = clean_datafram(orders_df,"Orders")
orders_df = get_volume_utilization(orders_df)

display(orders_df)

silver_path = f"abfss://{processed_container}@{storage_account_name}.dfs.core.windows.net/foot_cost.parquet"
orders_df.write.mode("overwrite").parquet(silver_path)


Cleaning Orders


product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,Space_utilization
a5bc1334f1762ce0a844f3f694dc08e1,cama_mesa_banho,48,645,1,1200,16,10,16,2560.0
53f92b0474f91fcb5bd188c6a8075c38,utilidades_domesticas,54,2952,3,30000,76,51,51,197676.0
45852eb1811c96c22787d9a3cb3875dc,cama_mesa_banho,56,393,2,1100,35,9,35,11025.0
bc7496ccc90a64c022a56021c65c9cda,utilidades_domesticas,28,238,1,150,26,16,16,6656.0
feddfbd2585f9007f7ed77b1d1ae8d80,beleza_saude,54,2627,1,700,16,16,15,3840.0
aaea62612b84679ddf91cbdda1183e7e,utilidades_domesticas,36,2600,1,850,43,5,24,5160.0
485c67ced40028609fbe77043fa0b943,utilidades_domesticas,47,469,1,9650,22,52,42,48048.0
02fcaaaba3a4b4882987a3639d968572,cama_mesa_banho,55,139,3,950,36,6,27,5832.0
a1b156a0925874f584d8d0cb45a962ba,utilidades_domesticas,59,1115,1,14600,43,62,41,109306.0
03e7d29d9988a30815f1f20b6549881f,fashion_calcados,54,404,4,700,22,12,33,8712.0
